In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import time
import os
import glob
from skimage import io, transform

from sklearn import tree
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

learn = tf.contrib.learn

In [ ]:
path = './data/data/'
w = 512
h = 512
c = 3

In [ ]:
def read_img(path):
    cate = [path + x for x in os.listdir(path) if os.path.isdir(path + x)]
    imgs = []
    cls = []
    labels = []
    
    classname = [x for x in os.listdir(path) if os.path.isdir(path + x)]
    
    for idx, folder in enumerate(cate):
        for im in glob.glob(folder + '/*.jpg'):
            print('reading the images:%s' % (im))
            single_label = [0 for x in range(8)]
            single_label[idx] = 1;
            img = io.imread(im)
            img = transform.resize(img, (w, h, c))
            featurevector = img.flatten()
            imgs.append(featurevector)
            cls.append(idx)
            labels.append(single_label)
    return np.asarray(imgs, np.float32), np.asarray(cls, np.int32), classname, np.asarray(labels, np.int32)

In [ ]:
data, cls,classnames,labels = read_img(path)

NameError: name 'read_img' is not defined

In [5]:
num_example = data.shape[0]
arr = np.arange(num_example)
np.random.shuffle(arr)
data = data[arr]
cls = cls[arr]
labels=labels[arr]

In [ ]:
ratio = 0.8
s = np.int(num_example * ratio)
images_train = data[:s]
images_test = data[s:]
cls_train = cls[:s]
cls_test = cls[s:]
labels_train = labels[:s]
labels_test = labels[s:]

num_classes = 8

In [ ]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(images_train)))
print("- Test-set:\t\t{}".format(len(images_test)))

Size of:
- Training-set:		3816
- Test-set:		955


In [ ]:
ESTIMATORS = {
    "Linear regression": LinearRegression(),
    "SVC": SVC(),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": tree.DecisionTreeClassifier(),
}

In [ ]:
cls_test_predict = dict()
for name, estimator in ESTIMATORS.items():
    print("estimator: {} fit start".format(name))
    estimator.fit(images_train, cls_train)
    print("estimator: {} fit complete".format(name))
    cls_test_predict[name] = estimator.predict(images_test)
    print("predict_complete: {}".format(name))

estimator: Linear regression fit start
estimator: Linear regression fit complete
predict_complete: Linear regression
estimator: SVC fit start


In [ ]:
def predict_cls(cls_true,cls_predict):
    correct = (cls_true==cls_pred)
    return correct.mean()

In [ ]:
for name in ESTIMATORS.items():
    predict = predict_cls(cls_test,cls_test_predict[name])
    print("{0}: {1:.1%}".format(name,predict)